# Feature Engineering "transactions"
<img src="https://cdn2.iconfinder.com/data/icons/webstore/512/dollar_money_bag-512.png" border="1" alt="Dataframe transactions" width="200" height="150">

This Python Notebook generates features for "transactions.csv" or "transactions_v2.csv" (depending if you want to obtain the file for training, until March 2017, or for testing, until April 2017). Later, it exports this wanted file into a folder called data. 

I will explain which lines you need to change but writing "TO BE CHANGED IF NEEDED" in Python comments in the line above.

In this file, there are:
- Features have been removed after checking that they did not improve the score (maybe due to **overfitting**).
- Features added and hot-encoded so that XGBoost has a better performance.
- Group the features by "msno" and add its values so that we only keep 1 row - 1 msno.

In the notebook "churn_prediction" then I will load "final_transactions" to create the prediction.

In [1]:
#Import the relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpld3
import seaborn as sns
import matplotlib.dates as mdates
import time
import datetime
from datetime import datetime as dt
from pandas.lib import Timestamp

#Configure Panda
pd.options.display.width = 200

/Users/pablodepaz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  # This is added back by InteractiveShellApp.init_path()


## 1. Data import and Feature Engineering

In [2]:
#TO BE CHANGED IF NEEDED

#Load transactions dataframe (use this if you want to predict for April 2017)
#trans = pd.read_csv("data/transactions.csv")
#trans_v2 = pd.read_csv("data/transactions_v2.csv")
#print('Data loaded!')
#transactions = pd.concat([trans, trans_v2])
#print('Concat done!')

#Load transactions dataframe (use this if you want to predict for March 2017)
transactions = pd.read_csv("data/transactions.csv")
print('Data loaded!')

Data loaded!


In [3]:
#Look at the first values in transactions:
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0


In [4]:
print(transactions.shape)

(21547746, 9)


In [5]:
#Remove rows whose transaction_date is in February or more!
#Use transactions_v1 and train_v1
transactions = transactions.query('transaction_date < 20170201')
#I TAKE OUT 908263 VALUES WHOSE TRANSACTION DATE IS IN FEBRUARY OR ON!

In [6]:
#transactions.head()

In [8]:
#Look at number of unique values in each feature:
print(transactions.nunique())
print('')
print('Number of rows & columns "TRANSACTIONS": ', transactions.shape)

msno                      2330992
payment_method_id              40
payment_plan_days              37
plan_list_price                51
actual_amount_paid             57
is_auto_renew                   2
transaction_date              762
membership_expire_date       1559
is_cancel                       2
dtype: int64

Number of rows & columns "TRANSACTIONS":  (20639483, 9)


There are 9 columns. One of them is MSNO so, there are 8 features.
<br/> Next, I will:
- Change dates to _dtype_ to be able to operate with them.
- Compute "maximum expiration date" for each msno
- Feature Engineering

### 1.1. Change dates to _dype_
It takes around 12 min

In [9]:
transactions['transaction_date_dtype'] = transactions.transaction_date.apply(lambda x: dt.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN" )
transactions['membership_expire_date_dtype'] = transactions.membership_expire_date.apply(lambda x: dt.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN" )
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_date_dtype,membership_expire_date_dtype
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0,2015-09-30,2015-11-01
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0,2015-09-30,2015-10-31
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0,2015-09-30,2016-04-27
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0,2015-09-30,2015-11-28
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0,2015-09-30,2015-11-21


### <font color='red'>Now, for each feature I need to compute by **grouping** the sum of the value the feature is related to.</font>

### 1.2. "Dataframe" and "new file": max_expire_date (for a given _msno_)
The following line of code normally takes like 1h - 1:30h to run! Be patient...
<font color='red'>UPDATE:</font> **I crated this file for:**
- 1) Whole file transactions_v2 **(max_expire_date_df_v2_all_file.csv)**
- 2) Concat file of transactions and transactions_v2 **(max_expire_date_df_v1_v2.csv)**
- 3) Whole file of transactions **(max_expire_date_df_v1.csv)**
- 4) 3M rows of transactions and whole transactions_v2 **(max_expire_date_df_v1_v2_3M_rows.csv)**. This one was for testing.
<br/><br/>You do not need to compute it again, so I just put it as "plain text"

max_expire_date_df = transactions.groupby('msno', as_index=False)['membership_expire_date_dtype'].max()
print('Done!')

#New dataframe
max_expire_date_df.head()
#print(max_expire_date_df.shape)
#print(max_expire_date_df.count())
#print(max_expire_date_df.nunique()) #It is correct!

I will save this as a new .csv file to avoid computing it in the future (as it takes a lot of time).

#TO BE CHANGED IF NEEDED
max_expire_date_df.to_csv('data/max_expire_date_df_v1_feb.csv', index = False)
print('Done :)')

<img src="https://www.calshrm.org/images/soft%20slit%20separator.png?crc=4104113101" border="1" alt="Dataframe transactions" width="400" height="100">


### 1.3. <font color='red'>"Feature"</font>: membership duration (in total for al records of the same user) <font color='red'>mem_duration</font>

Difference between transaction_date and membership_expire_date. The difference is in terms of days (integer).

Later, I will compute the sum of all for each user (at the end of this file) by grouping them!

This line takes around 5 min

In [13]:
#--- difference in days ---
transactions['mem_duration'] = transactions.membership_expire_date_dtype - transactions.transaction_date_dtype
transactions['mem_duration'] = transactions['mem_duration'] / np.timedelta64(1, 'D')
transactions['mem_duration'] = transactions['mem_duration'].astype(int)

In [14]:
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_date_dtype,membership_expire_date_dtype,mem_duration
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0,2015-09-30,2015-11-01,32
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0,2015-09-30,2015-10-31,31
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0,2015-09-30,2016-04-27,210
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0,2015-09-30,2015-11-28,59
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0,2015-09-30,2015-11-21,52


In [15]:
print(transactions.mem_duration.unique())
print('')
print(transactions.mem_duration.describe())

[    32     31    210 ... -16636   -173 -16589]

count    2.063948e+07
mean     3.610140e+01
std      1.479675e+02
min     -1.719700e+04
25%      3.000000e+01
50%      3.100000e+01
75%      3.100000e+01
max      8.140000e+02
Name: mem_duration, dtype: float64


**The following is a graph but I just put it as a text to ease the reading of the whole document**

sns.set(rc={'figure.figsize':(13,4)}) #Horizontal, vertical
sns.distplot( transactions["mem_duration"], bins = 1000 )
plt.ylabel('Count', fontsize = 12)
plt.xlabel('Membership duration', fontsize = 12)
plt.show()

I can see that 30 is the most general membership duration. So I will put as 30 the values above 100.
I will one-hot encode this and get rid of the columns less important.

In [16]:
#Change the mem_duration of all rows with an value greater than 60 to 40
#https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/
transactions.loc[transactions['mem_duration'] > 60, "mem_duration"] = 30
transactions.loc[transactions['mem_duration'] < 0, "mem_duration"] = 0

**The following is a graph but I just put it as a text to ease the reading of the whole document**

sns.set(rc={'figure.figsize':(13,4)}) #Horizontal, vertical
sns.distplot( transactions["mem_duration"], bins = 500 )
plt.ylabel('Count', fontsize = 12)
plt.xlabel('Membership duration', fontsize = 12)
plt.show()

In [17]:
transactions.mem_duration.unique()

array([32, 31, 30, 59, 52, 38, 56, 49, 55, 48, 60, 42, 53, 37, 45, 54, 39,
       44, 50,  8, 35, 34, 10, 36, 57, 47, 58, 46, 40, 33, 29, 28, 51, 41,
        0, 43,  7,  5,  1,  6, 14,  4, 27, 13, 11, 16, 12, 21,  3, 26,  2,
       24, 15,  9, 19, 18, 20, 23, 25, 17, 22])

In [18]:
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_date_dtype,membership_expire_date_dtype,mem_duration
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0,2015-09-30,2015-11-01,32
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0,2015-09-30,2015-10-31,31
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0,2015-09-30,2016-04-27,30
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0,2015-09-30,2015-11-28,59
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0,2015-09-30,2015-11-21,52


### 1.4. <font color='red'>"Feature Eng."</font>: _trans1_. Hot-encode "payment_method_id" feature <font color='red'>payment_method_id_XX</font>

In [19]:
transactions.payment_method_id.unique()

array([41, 39, 21, 37, 40, 34, 33, 31, 23, 38, 19, 24, 14, 36, 27, 35, 29,
       30, 28, 22, 26, 32, 25, 20, 17,  5, 11, 18,  7, 13, 12, 16, 10, 15,
        8,  6,  3,  2,  4,  1])

In [20]:
#To improve the speed of running this notebook, I put all values between 0 and 29 (inclusive) to have value 1.
#These are not important payment method values.
transactions['payment_method_id'] = np.where(transactions['payment_method_id'].between(0,29), 1, transactions['payment_method_id']) #Replace by 1's the values in the range

In [21]:
transactions.payment_method_id.unique()

array([41, 39,  1, 37, 40, 34, 33, 31, 38, 36, 35, 30, 32])

In [22]:
trans1 = transactions

#One-hot encode payment_method_id and save it into payment_method_id_encode
payment_method_id_encode = pd.get_dummies(trans1['payment_method_id'], prefix='payment_method_id')
print('Done!')

#Join the encoded payment_method_id_encode
trans1 = trans1.join(payment_method_id_encode)
print('Done!')

Done!
Done!


In [23]:
#I drop the column of value 1, which at the same time has all the values between 0 and 20 (as it was changed 3 blocks of code above)
print('Start...')
trans1 = trans1.drop("payment_method_id_1", axis=1)
print('Done!')

Start...
Done!


In [24]:
trans1.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_date_dtype,...,payment_method_id_32,payment_method_id_33,payment_method_id_34,payment_method_id_35,payment_method_id_36,payment_method_id_37,payment_method_id_38,payment_method_id_39,payment_method_id_40,payment_method_id_41
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0,2015-09-30,...,0,0,0,0,0,0,0,0,0,1
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0,2015-09-30,...,0,0,0,0,0,0,0,0,0,1
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0,2015-09-30,...,0,0,0,0,0,0,0,0,0,1
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0,2015-09-30,...,0,0,0,0,0,0,0,1,0,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0,2015-09-30,...,0,0,0,0,0,0,0,1,0,0


In [25]:
print('Number of rows & columns: ', trans1.shape)

Number of rows & columns:  (20639483, 24)


### 1.5. <font color='red'>"Feature Eng."</font>: <font color='red'>pay_method_churn</font>, create 3 variables for payment method 32, 35, 38 
These 3 payment methods are the ones that have most probability to _churn_ as can be seen in the data exploration

In [26]:
mask = (trans1.payment_method_id == 32)
mask1 = (trans1.payment_method_id == 35)
mask2 = (trans1.payment_method_id == 38)

trans1['pay_method_churn'] = trans1.payment_method_id
trans1.pay_method_churn = 0
column_name = 'pay_method_churn'
trans1.loc[mask,column_name] = 1
trans1.loc[mask1,column_name] = 1
trans1.loc[mask2,column_name] = 1

trans1 = trans1.drop('payment_method_id', axis = 1)

trans1.head()

,msno,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_date_dtype,membership_expire_date_dtype,...,payment_method_id_33,payment_method_id_34,payment_method_id_35,payment_method_id_36,payment_method_id_37,payment_method_id_38,payment_method_id_39,payment_method_id_40,payment_method_id_41,pay_method_churn
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,30,129,129,1,20150930,20151101,0,2015-09-30,2015-11-01,...,0,0,0,0,0,0,0,0,1,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,30,149,149,1,20150930,20151031,0,2015-09-30,2015-10-31,...,0,0,0,0,0,0,0,0,1,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,30,129,129,1,20150930,20160427,0,2015-09-30,2016-04-27,...,0,0,0,0,0,0,0,0,1,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,30,149,149,1,20150930,20151128,0,2015-09-30,2015-11-28,...,0,0,0,0,0,0,1,0,0,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,30,149,149,1,20150930,20151121,0,2015-09-30,2015-11-21,...,0,0,0,0,0,0,1,0,0,0


### 1.6. <font color='red'>"Feature Eng."</font>: "trans2". Hot-encode "payment_plan_days" feature and test algorithm <font color='red'>payment_plan_days_XX</font>

In [27]:
trans1.payment_plan_days.unique()

array([ 30,   0,  10,  31,   7, 410, 195, 100, 395,   1, 180, 120,  60,
       400,  14, 360, 200,  35,  90,  21, 450,  70,  80,  45, 110, 365,
        66, 240, 270,  99,   2, 230,   3,  15, 425, 415,  95])

In [28]:
#All outliers (not important values) are changed its value to 1. Later, this value will be dropped!
trans1['payment_plan_days'] = np.where(trans1['payment_plan_days'].between(0,3), 1, trans1['payment_plan_days']) #Replace by 1's the values in the range
trans1['payment_plan_days'] = np.where(trans1['payment_plan_days'].between(8,29), 1, trans1['payment_plan_days']) #Replace by 1's the values in the range
trans1['payment_plan_days'] = np.where(trans1['payment_plan_days'].between(31,89), 1, trans1['payment_plan_days']) #Replace by 1's the values in the range
print('Done1')
trans1['payment_plan_days'] = np.where(trans1['payment_plan_days'].between(95,179), 1, trans1['payment_plan_days']) #Replace by 1's the values in the range
trans1['payment_plan_days'] = np.where(trans1['payment_plan_days'].between(181,194), 1, trans1['payment_plan_days']) #Replace by 1's the values in the range
print('Done2')
trans1['payment_plan_days'] = np.where(trans1['payment_plan_days'].between(196,409), 1, trans1['payment_plan_days']) #Replace by 1's the values in the range
trans1['payment_plan_days'] = np.where(trans1['payment_plan_days'].between(411,500), 1, trans1['payment_plan_days']) #Replace by 1's the values in the range

Done1
Done2


In [29]:
trans1.payment_plan_days.unique()

array([ 30,   1,   7, 410, 195, 180,  90])

In [30]:
trans2 = trans1

payment_plan_days_encode = pd.get_dummies(trans2['payment_plan_days'], prefix='payment_plan_days')

trans2 = trans2.drop('payment_plan_days', axis=1)

print('Start')
trans2 = trans2.join(payment_plan_days_encode)
print('Done2')
#trans2.head()

Start
Done2


In [31]:
#I get rid of columns whose value is 1
print('Start!')
trans2 = trans2.drop("payment_plan_days_1", axis=1)
print('Done!')

Start!
Done!


In [32]:
trans2.head()

,msno,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_date_dtype,membership_expire_date_dtype,mem_duration,...,payment_method_id_39,payment_method_id_40,payment_method_id_41,pay_method_churn,payment_plan_days_7,payment_plan_days_30,payment_plan_days_90,payment_plan_days_180,payment_plan_days_195,payment_plan_days_410
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,129,129,1,20150930,20151101,0,2015-09-30,2015-11-01,32,...,0,0,1,0,0,1,0,0,0,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,149,149,1,20150930,20151031,0,2015-09-30,2015-10-31,31,...,0,0,1,0,0,1,0,0,0,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,129,129,1,20150930,20160427,0,2015-09-30,2016-04-27,30,...,0,0,1,0,0,1,0,0,0,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,149,149,1,20150930,20151128,0,2015-09-30,2015-11-28,59,...,1,0,0,0,0,1,0,0,0,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,149,149,1,20150930,20151121,0,2015-09-30,2015-11-21,52,...,1,0,0,0,0,1,0,0,0,0


In [33]:
print('Number of rows & columns: ', trans2.shape)

Number of rows & columns:  (20639483, 29)


There is no 1.7 point
### 1.8. <font color='red'>"Feature Eng."</font>: notAutorenew_&_cancel
Binary feature to predict possible churning if
- auto_renew = 0 and
- is_cancel = 1

In [34]:
trans2['notAutorenew_and_cancel'] = ((trans2.is_auto_renew == 0) == (trans2.is_cancel == 1)).astype(np.int8)

In [35]:
trans2.head()

,msno,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,transaction_date_dtype,membership_expire_date_dtype,mem_duration,...,payment_method_id_40,payment_method_id_41,pay_method_churn,payment_plan_days_7,payment_plan_days_30,payment_plan_days_90,payment_plan_days_180,payment_plan_days_195,payment_plan_days_410,notAutorenew_and_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,129,129,1,20150930,20151101,0,2015-09-30,2015-11-01,32,...,0,1,0,0,1,0,0,0,0,1
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,149,149,1,20150930,20151031,0,2015-09-30,2015-10-31,31,...,0,1,0,0,1,0,0,0,0,1
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,129,129,1,20150930,20160427,0,2015-09-30,2016-04-27,30,...,0,1,0,0,1,0,0,0,0,1
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,149,149,1,20150930,20151128,0,2015-09-30,2015-11-28,59,...,0,0,0,0,1,0,0,0,0,1
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,149,149,1,20150930,20151121,0,2015-09-30,2015-11-21,52,...,0,0,0,0,1,0,0,0,0,1


### 1.9. <font color='red'>"GROUPBY ALL COLUMNS"</font>
I first drop unwanted features and then, I group all **msno**! (obtaining a dataframe with unique msnos)

In [36]:
#DATAFRAME I WILL WORK WITH
unwanted = ['transaction_date_dtype','membership_expire_date_dtype','plan_list_price','actual_amount_paid','transaction_date','membership_expire_date']
trans2 = trans2.drop(unwanted, axis = 1)

The following line takes around 5 min...

In [37]:
sums_df = trans2.groupby('msno').sum().reset_index()

In [38]:
sums_df.head()

,msno,is_auto_renew,is_cancel,mem_duration,payment_method_id_30,payment_method_id_31,payment_method_id_32,payment_method_id_33,payment_method_id_34,payment_method_id_35,...,payment_method_id_40,payment_method_id_41,pay_method_churn,payment_plan_days_7,payment_plan_days_30,payment_plan_days_90,payment_plan_days_180,payment_plan_days_195,payment_plan_days_410,notAutorenew_and_cancel
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,0,0,5,0,0,0,0,0,1,...,0,0,1,1,0,0,0,0,0,0
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0,0,30,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,3,0,91,0,0,0,0,0,0,...,0,3,0,0,3,0,0,0,0,3
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,0,932,0,0,0,0,0,0,...,0,0,0,0,15,0,0,0,0,19
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,25,0,762,0,0,0,0,0,0,...,0,25,0,0,24,0,0,0,0,25


In [39]:
print(sums_df.shape)

(2330992, 24)


**Now, I have a dataframe with all my data grouped by msno and added values of each feature (SUM)**

### 1.10. <font color='red'>"Feature Eng."</font>: cancel_ratio
Next code box will take 4 mins.

In [40]:
counts = trans2.groupby('msno')['is_cancel'].count().reset_index()
counts.columns = ['msno','transactions']
sums_is_cancel = trans2.groupby('msno')['is_cancel'].sum().reset_index()
merged_dataset = sums_is_cancel.merge(counts, how='inner', on='msno')

merged_dataset['is_cancel_number'] = merged_dataset['is_cancel']
merged_dataset = merged_dataset.drop(['is_cancel'], axis = 1)
print(merged_dataset.shape)

(2330992, 3)


In [41]:
merged_dataset.head()

,msno,transactions,is_cancel_number
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,1,0
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,1,0
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,3,0
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,0
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,25,0


### 1.11. Merge step 1.9. and 1.10.

In [42]:
result = sums_df.merge(merged_dataset, on='msno')
print(result.shape)

(2330992, 26)


In [43]:
result.head()

,msno,is_auto_renew,is_cancel,mem_duration,payment_method_id_30,payment_method_id_31,payment_method_id_32,payment_method_id_33,payment_method_id_34,payment_method_id_35,...,pay_method_churn,payment_plan_days_7,payment_plan_days_30,payment_plan_days_90,payment_plan_days_180,payment_plan_days_195,payment_plan_days_410,notAutorenew_and_cancel,transactions,is_cancel_number
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,0,0,5,0,0,0,0,0,1,...,1,1,0,0,0,0,0,0,1,0
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0,0,30,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,3,0,91,0,0,0,0,0,0,...,0,0,3,0,0,0,0,3,3,0
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,0,932,0,0,0,0,0,0,...,0,0,15,0,0,0,0,19,19,0
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,25,0,762,0,0,0,0,0,0,...,0,0,24,0,0,0,0,25,25,0


### 1.12. Put together 1.11. and the file I created in the first parts of this notebook: "max_expire_date_df.csv" (part 1.2.)

In [44]:
#CHECK IF YOU HAVE THIS FILE! If not, email pablo.depaz@hotmail.es asking for help :)

#TO BE CHANGED IF NEEDED
max_expire_date_dataframe = pd.read_csv('data/max_expire_date_df_v1_feb.csv')
print('Done!')

Done!


**Now, merge this with the "result" dataframe in section 1.11.**

In [45]:
merging = result.merge(max_expire_date_dataframe, on='msno')
print(merging.shape)

(2330992, 27)


In [46]:
merging.head()

,msno,is_auto_renew,is_cancel,mem_duration,payment_method_id_30,payment_method_id_31,payment_method_id_32,payment_method_id_33,payment_method_id_34,payment_method_id_35,...,payment_plan_days_7,payment_plan_days_30,payment_plan_days_90,payment_plan_days_180,payment_plan_days_195,payment_plan_days_410,notAutorenew_and_cancel,transactions,is_cancel_number,membership_expire_date_dtype
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,0,0,5,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,2016-09-14
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0,0,30,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,2017-01-04
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,3,0,91,0,0,0,0,0,0,...,0,3,0,0,0,0,3,3,0,2017-02-15
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,0,932,0,0,0,0,0,0,...,0,15,0,0,0,0,19,19,0,2017-03-19
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,25,0,762,0,0,0,0,0,0,...,0,24,0,0,0,0,25,25,0,2017-02-26


### 1.13. Check if missing values

In [47]:
merging.isnull().sum()
#We should get 0!

msno                            0
is_auto_renew                   0
is_cancel                       0
mem_duration                    0
payment_method_id_30            0
payment_method_id_31            0
payment_method_id_32            0
payment_method_id_33            0
payment_method_id_34            0
payment_method_id_35            0
payment_method_id_36            0
payment_method_id_37            0
payment_method_id_38            0
payment_method_id_39            0
payment_method_id_40            0
payment_method_id_41            0
pay_method_churn                0
payment_plan_days_7             0
payment_plan_days_30            0
payment_plan_days_90            0
payment_plan_days_180           0
payment_plan_days_195           0
payment_plan_days_410           0
notAutorenew_and_cancel         0
transactions                    0
is_cancel_number                0
membership_expire_date_dtype    0
dtype: int64

<img src="https://www.calshrm.org/images/soft%20slit%20separator.png?crc=4104113101" border="1" alt="Dataframe transactions" width="400" height="100">


# 2. Create 3 dataframes with train_v1, train_v2 and submission_file.
I compute the days difference between **max_expire_date** and:
- Feb 28th (**for train_v1**)
- March 31th (**for train_v2**). We use this one
- April 30th (**for submission_file**). We use this one
<img src="http://tripkendall.com/wp-content/uploads/2018/01/pandas_logo-1080x675.jpg" border="1" alt="Dataframe transactions" width="200" height="150">

In [48]:
#Import train sets and sample_submission_zero.csv
train = pd.read_csv('data/train.csv')
print(train.shape)
train_v2 = pd.read_csv('data/train_v2.csv')
print(train_v2.shape)
sample_sub = pd.read_csv('data/sample_submission_v2.csv')
print(sample_sub.shape)

(992931, 2)
(970960, 2)
(907471, 2)


### 2.1. Compute the dates difference

In [49]:
#Compute the difference for "train" (w.r.t. 2017 Feb 28th)
merging['membership_expire_date_dtype'] = pd.to_datetime(merging['membership_expire_date_dtype'])
merging.dtypes

merging['diff_feb'] = merging['membership_expire_date_dtype'] - datetime.date(2017,2,28)
merging['diff_feb'] = merging['diff_feb'] / np.timedelta64(1, 'D')
merging['diff_feb'] = merging['diff_feb'].astype(int)

In [50]:
#Compute the difference for "train" (w.r.t. 2017 March 31st)
merging['diff_mar'] = merging['membership_expire_date_dtype'] - datetime.date(2017,3,31)
merging['diff_mar'] = merging['diff_mar'] / np.timedelta64(1, 'D')
merging['diff_mar'] = merging['diff_mar'].astype(int)

In [51]:
#Compute the difference for "train" (w.r.t. 2017 April 30th)
merging['diff_apr'] = merging['membership_expire_date_dtype'] - datetime.date(2017,4,30)
merging['diff_apr'] = merging['diff_apr'] / np.timedelta64(1, 'D')
merging['diff_apr'] = merging['diff_apr'].astype(int)

In [52]:
#I don't need it anymore
merging = merging.drop("membership_expire_date_dtype", axis = 1)

So, here I have in **merging** 3 features:
- **"diff_feb"**: feature with the difference of the expiration date w.r.t. 2017 Feb 28th.
- **"diff_mar"**: feature with the difference of the expiration date w.r.t. 2017 Mar 31st.
- **"diff_apr"**: feature with the difference of the expiration date w.r.t. 2017 Apr 30th.

### 2.2. Merge the "dfX" dataframes with train, train_v2 and sample_submission

In [53]:
#TO BE CHANGED IF NEEDED (change right file to merge with)
train_final = pd.merge(merging, train, on = 'msno', how = 'right')

#TO BE CHANGED IF NEEDED (change the features to drop)
train_final = train_final.drop("diff_apr", axis=1)
train_final = train_final.drop("diff_mar", axis=1)

In [54]:
#Rename the feature of the difference with the same name! (to be able to work with different files)

#TO BE CHANGED IF NEEDED (change the feature in the right part of the assignment below)
train_final['diff'] = train_final['diff_feb']

In [55]:
#TO BE CHANGED IF NEEDED (drop the feature and keep the one with the "diff" name)
train_final = train_final.drop("diff_feb", axis=1)
train_final.head()

,msno,is_auto_renew,is_cancel,mem_duration,payment_method_id_30,payment_method_id_31,payment_method_id_32,payment_method_id_33,payment_method_id_34,payment_method_id_35,...,payment_plan_days_30,payment_plan_days_90,payment_plan_days_180,payment_plan_days_195,payment_plan_days_410,notAutorenew_and_cancel,transactions,is_cancel_number,is_churn,diff
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,3.0,0.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0,-13.0
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19.0,0.0,932.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,0.0,0.0,0.0,0.0,19.0,19.0,0.0,0,19.0
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,25.0,0.0,762.0,0.0,0.0,0.0,0.0,0.0,0.0,...,24.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0,0,-2.0
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,11.0,0.0,337.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.0,0.0,0.0,0.0,0.0,11.0,11.0,0.0,0,-13.0
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,19.0,0.0,1008.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,0.0,0.0,0.0,0.0,19.0,19.0,0.0,0,23.0


In [56]:
print(train_final.shape)
#print(train_final.describe().T)

(992931, 28)


### 2.3. Check for missing values

In [57]:
train_final.isnull().sum()
#For the missing values, I will just put the majority values for each feature

msno                          0
is_auto_renew              2097
is_cancel                  2097
mem_duration               2097
payment_method_id_30       2097
payment_method_id_31       2097
payment_method_id_32       2097
payment_method_id_33       2097
payment_method_id_34       2097
payment_method_id_35       2097
payment_method_id_36       2097
payment_method_id_37       2097
payment_method_id_38       2097
payment_method_id_39       2097
payment_method_id_40       2097
payment_method_id_41       2097
pay_method_churn           2097
payment_plan_days_7        2097
payment_plan_days_30       2097
payment_plan_days_90       2097
payment_plan_days_180      2097
payment_plan_days_195      2097
payment_plan_days_410      2097
notAutorenew_and_cancel    2097
transactions               2097
is_cancel_number           2097
is_churn                      0
diff                       2097
dtype: int64

**Some data they have missing values, if so: uncomment next 2 blocks of code and run them!**

In [58]:
values = {'is_auto_renew': 1, 'is_cancel': 0, 'mem_duration': 488, 'payment_method_id_30': 0, 'payment_method_id_31': 0, 'payment_method_id_32': 0, 'payment_method_id_33': 0, 'payment_method_id_34': 0, 'payment_method_id_35': 0, 'payment_method_id_36': 0, 'payment_method_id_37': 0, 'payment_method_id_38': 0, 'payment_method_id_39': 1, 'payment_method_id_40': 1, 'payment_method_id_41': 9, 'pay_method_churn': 0, 'payment_plan_days_7': 0, 'payment_plan_days_30': 14, 'payment_plan_days_90': 0, 'payment_plan_days_180': 0, 'payment_plan_days_195': 0, 'payment_plan_days_410': 0, 'notAutorenew_and_cancel': 0, 'transactions': 15, 'is_cancel_number': 0, 'diff': -14, 'is_churn': 0}
train_final = train_final.fillna(value = values)

In [59]:
train_final.isnull().sum()

msno                       0
is_auto_renew              0
is_cancel                  0
mem_duration               0
payment_method_id_30       0
payment_method_id_31       0
payment_method_id_32       0
payment_method_id_33       0
payment_method_id_34       0
payment_method_id_35       0
payment_method_id_36       0
payment_method_id_37       0
payment_method_id_38       0
payment_method_id_39       0
payment_method_id_40       0
payment_method_id_41       0
pay_method_churn           0
payment_plan_days_7        0
payment_plan_days_30       0
payment_plan_days_90       0
payment_plan_days_180      0
payment_plan_days_195      0
payment_plan_days_410      0
notAutorenew_and_cancel    0
transactions               0
is_cancel_number           0
is_churn                   0
diff                       0
dtype: int64

In [60]:
print(train_final.shape)
#Now I have the desired features (28 features)

(992931, 28)


<img src="https://www.calshrm.org/images/soft%20slit%20separator.png?crc=4104113101" border="1" alt="Dataframe transactions" width="400" height="100">


## 3. Export file
It should take around 2 minutes!

In [61]:
train_final.to_csv('data/transactions_feb.csv', index = False)
print('Done!')

Done!
